# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [19]:
!pip install --upgrade pandas

Requirement not upgraded as not directly required: python-dateutil>=2.6.1 in c:\users\abdullah\anaconda3\envs\tensorflowenv\lib\site-packages (from pandas) (2.7.3)
Requirement not upgraded as not directly required: numpy>=1.13.3 in c:\users\abdullah\anaconda3\envs\tensorflowenv\lib\site-packages (from pandas) (1.14.5)
Requirement not upgraded as not directly required: pytz>=2017.2 in c:\users\abdullah\anaconda3\envs\tensorflowenv\lib\site-packages (from pandas) (2018.4)
Requirement not upgraded as not directly required: six>=1.5 in c:\users\abdullah\anaconda3\envs\tensorflowenv\lib\site-packages (from python-dateutil>=2.6.1->pandas) (1.11.0)
  Found existing installation: pandas 0.23.3
    Uninstalling pandas-0.23.3:
      Successfully uninstalled pandas-0.23.3


tensorflow-gpu 1.8.0 has requirement tensorboard<1.9.0,>=1.8.0, but you'll have tensorboard 1.12.2 which is incompatible.
Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'C:\\Users\\Abdullah\\AppData\\Local\\Temp\\pip-uninstall-i7v4aa2t\\users\\abdullah\\anaconda3\\envs\\tensorflowenv\\lib\\site-packages\\pandas\\io\\msgpack\\_packer.cp35-win_amd64.pyd'
Consider using the `--user` option or check the permissions.

You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [72]:
# import libraries
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine

import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords

from sklearn.multioutput import MultiOutputClassifier

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score , f1_score,  precision_score, recall_score
from sklearn.model_selection import train_test_split
import pickle
from sklearn.model_selection import GridSearchCV

In [37]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Abdullah\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Abdullah\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Abdullah\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [38]:
# load data from database
engine = create_engine('sqlite:///disaster_database.db')
df = pd.read_sql_table('disaster_messages',engine.connect())

In [39]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
X = df["message"]
y = df.drop(columns=["id",'message',"original","genre"])
label_names = y.columns
y=y

### 2. Write a tokenization function to process your text data

In [41]:
def tokenize(text):
    lemmatizer = WordNetLemmatizer() 
    #remove punctuation and uwanted characters
    text=re.sub("r[^a-zA-Z0-9]"," ",text)
    #tokenzie the words
    words= nltk.word_tokenize(text)
    #lower case the words and lemmantize them
    words = [lemmatizer.lemmatize(word.lower()) for word in words]
    #remove stop words
    words= [word for word in words if word not in stopwords.words('english')]
    
    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [77]:
pipeline = Pipeline([
    ('bow',CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('classifier',MultiOutputClassifier(KNeighborsClassifier(n_jobs=-1))) 
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [43]:


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [44]:
clf = pipeline.fit(X_train,y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [48]:
ypred= clf.predict(X_test)

In [63]:
print(classification_report(np.array(y_test,ndmin=2),np.array(ypred,ndmin=2), labels=list(range(36)),target_names= label_names ))

                        precision    recall  f1-score   support

               related       0.85      0.89      0.87      4020
               request       0.66      0.55      0.60       908
                 offer       0.00      0.00      0.00        21
           aid_related       0.67      0.59      0.62      2205
          medical_help       0.46      0.15      0.23       416
      medical_products       0.67      0.25      0.36       267
     search_and_rescue       0.57      0.08      0.14       163
              security       0.00      0.00      0.00        86
              military       0.78      0.12      0.21       169
           child_alone       0.00      0.00      0.00         0
                 water       0.63      0.35      0.45       311
                  food       0.64      0.40      0.50       603
               shelter       0.62      0.30      0.40       471
              clothing       0.60      0.28      0.38        86
                 money       0.48      

C:\Users\Abdullah\Anaconda3\envs\tensorflowEnv\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Abdullah\Anaconda3\envs\tensorflowEnv\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Abdullah\Anaconda3\envs\tensorflowEnv\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
C:\Users\Abdullah\Anaconda3\envs\tensorflowEnv\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with

### 6. Improve your model
Use grid search to find better parameters. 

In [86]:
parameters = {
    "classifier__estimator__n_neighbors":[3,5,7],
    "classifier__estimator__weights":["uniform","distance"]
}

cv = GridSearchCV(clf, parameters)

In [87]:
cv.fit(X_train,y_train)

C:\Users\Abdullah\Anaconda3\envs\tensorflowEnv\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('bow',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                        

In [88]:
knn_best_estimator = cv.best_estimator_

In [90]:
filename = 'knn_best_model.sav'
pickle.dump(knn_best_estimator, open(filename, 'wb'))

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [91]:
ypred_knn_best= knn_best_estimator.predict(X_test)

In [92]:
print(classification_report(np.array(y_test,ndmin=2),np.array(ypred_knn_best,ndmin=2), labels=list(range(36)),target_names= label_names ))

                        precision    recall  f1-score   support

               related       0.85      0.89      0.87      4020
               request       0.66      0.55      0.60       908
                 offer       0.00      0.00      0.00        21
           aid_related       0.67      0.59      0.62      2205
          medical_help       0.46      0.15      0.23       416
      medical_products       0.67      0.25      0.36       267
     search_and_rescue       0.57      0.08      0.14       163
              security       0.00      0.00      0.00        86
              military       0.78      0.12      0.21       169
           child_alone       0.00      0.00      0.00         0
                 water       0.63      0.35      0.45       311
                  food       0.64      0.40      0.50       603
               shelter       0.62      0.30      0.40       471
              clothing       0.60      0.28      0.38        86
                 money       0.48      

C:\Users\Abdullah\Anaconda3\envs\tensorflowEnv\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Abdullah\Anaconda3\envs\tensorflowEnv\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Abdullah\Anaconda3\envs\tensorflowEnv\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
C:\Users\Abdullah\Anaconda3\envs\tensorflowEnv\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [50]:
from sklearn.linear_model import LogisticRegression

In [52]:
pipeline_2 = Pipeline([
    ('bow',CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('classifier',LogisticRegression( multi_class="multinomial",n_job=-1) ) 
])

In [53]:
logitstic_regression = pipeline.fit(X_train,y_train)

In [54]:
ypred_logistic_regression = logitstic_regression.predict(X_test)

In [62]:
print(classification_report(np.array(y_test,ndmin=2),np.array(ypred_logistic_regression,ndmin=2), labels=list(range(36)),target_names= label_names ))

                        precision    recall  f1-score   support

               related       0.85      0.89      0.87      4020
               request       0.66      0.55      0.60       908
                 offer       0.00      0.00      0.00        21
           aid_related       0.67      0.59      0.62      2205
          medical_help       0.46      0.15      0.23       416
      medical_products       0.67      0.25      0.36       267
     search_and_rescue       0.57      0.08      0.14       163
              security       0.00      0.00      0.00        86
              military       0.78      0.12      0.21       169
           child_alone       0.00      0.00      0.00         0
                 water       0.63      0.35      0.45       311
                  food       0.64      0.40      0.50       603
               shelter       0.62      0.30      0.40       471
              clothing       0.60      0.28      0.38        86
                 money       0.48      

C:\Users\Abdullah\Anaconda3\envs\tensorflowEnv\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Abdullah\Anaconda3\envs\tensorflowEnv\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Abdullah\Anaconda3\envs\tensorflowEnv\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
C:\Users\Abdullah\Anaconda3\envs\tensorflowEnv\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with

In [69]:
filename = 'ligisitc_regression_model.sav'
pickle.dump(logitstic_regression, open(filename, 'wb'))

In [56]:
from sklearn.neural_network import MLPClassifier

In [58]:
pipeline_MLP = Pipeline([
    ('bow',CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('classifier',MLPClassifier() ) 
])

In [59]:
MLP = pipeline_MLP.fit(X_train,y_train)

C:\Users\Abdullah\Anaconda3\envs\tensorflowEnv\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [60]:
ypred_MLP = MLP.predict(X_test)

In [61]:
print(classification_report(np.array(y_test,ndmin=2),np.array(ypred_MLP,ndmin=2), labels=list(range(36)),target_names= label_names ))

                        precision    recall  f1-score   support

               related       0.85      0.86      0.86      4020
               request       0.57      0.60      0.59       908
                 offer       0.33      0.10      0.15        21
           aid_related       0.65      0.64      0.65      2205
          medical_help       0.35      0.32      0.34       416
      medical_products       0.47      0.33      0.38       267
     search_and_rescue       0.42      0.25      0.32       163
              security       0.06      0.02      0.03        86
              military       0.60      0.40      0.48       169
           child_alone       0.00      0.00      0.00         0
                 water       0.70      0.59      0.64       311
                  food       0.70      0.60      0.65       603
               shelter       0.64      0.53      0.58       471
              clothing       0.78      0.36      0.49        86
                 money       0.53      

C:\Users\Abdullah\Anaconda3\envs\tensorflowEnv\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Abdullah\Anaconda3\envs\tensorflowEnv\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Abdullah\Anaconda3\envs\tensorflowEnv\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
C:\Users\Abdullah\Anaconda3\envs\tensorflowEnv\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with

In [66]:
filename = 'MLP_model.sav'
pickle.dump(MLP, open(filename, 'wb'))

### 9. Export your model as a pickle file

In [68]:
filename = 'knn_model.sav'
pickle.dump(clf, open(filename, 'wb'))


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.